In [2]:
import json
import pandas as pd
import numpy as np
import re

In [3]:
## Testing with the first two instructions from the file
#decoded_instructions = []
#for hex_instr in hex_content[:1]:
#    for index, line in df_sorted.iterrows():
#        #print(line['bit_ranges'])
#        #line['bit_ranges']
#    hex_instr = hex_instr.strip()  # Removing newline characters
#    print(hex_to_bin(hex_instr))
#    #decoded_instructions.append(decode_instruction(hex_instr))
#
#decoded_instructions  # Display the decoded results

In [4]:
def hex_to_bin(hex_str):
    return (bin(int(hex_str, 16))[2:])

def decode_instruction(hex_instr):
    bin_instr = hex_to_bin(hex_instr) 
    opcode = bin_instr[-7:]
    instr_type = df[df['instruction']].get(opcode, 'Unknown')
    return bin_instr, opcode, instr_type

In [5]:
def parse_range(s: str):
    # Check if the string is in the 'X..Y=Z' format
    if '..' in s:
        # Use regex to extract X, Y, and Z
        match = re.match(r'(\d+)\.\.(\d+)=(-?0x[\da-fA-F]+|-?\d+)', s)
        if match:
            X = int(match.group(1))
            Y = int(match.group(2))
            Z = match.group(3)
            # Check if Z is hexadecimal, convert to binary if it is
            if Z.startswith('0x') or Z.startswith('-0x'):
                Z = hex_to_bin(Z)
            else:
                Z = bin(int(Z))[2:]  # Convert to binary if not hex
            return [X, Y, Z]
    else:
        # Use regex to extract X and Z in the 'X=Z' format
        match = re.match(r'(\d+)=(-?0x[\da-fA-F]+|-?\d+)', s)
        if match:
            X = int(match.group(1))
            Z = match.group(2)
            # Check if Z is hexadecimal, convert to binary if it is
            if Z.startswith('0x') or Z.startswith('-0x'):
                Z = hex_to_bin(Z)
            else:
                Z = bin(int(Z))[2:]  # Convert to binary if not hex
            return [X, Z]

In [6]:
json_file = './output_opcodes.json'

In [7]:
df = pd.read_json(json_file)
df.head()

,instruction,bit_ranges,extension
0,pseudo_op-slli,"[[31..25=0, 14..12=1, 6..2=0x04, 1..0=3]]",[rv32_i]
1,pseudo_op-srli,"[[31..25=0, 14..12=5, 6..2=0x04, 1..0=3]]",[rv32_i]
2,pseudo_op-srai,"[[31..25=32, 14..12=5, 6..2=0x04, 1..0=3]]",[rv32_i]
3,pseudo_op-slli_rv32,"[[31..25=0, 14..12=1, 6..2=0x04, 1..0=3]]",[rv32_i]
4,pseudo_op-srli_rv32,"[[31..25=0, 14..12=5, 6..2=0x04, 1..0=3]]",[rv32_i]


In [8]:
df['bit_ranges'] = df['bit_ranges'].apply(lambda x: np.array(x).ravel())

In [9]:
df[df['instruction'] == 'addi']

,instruction,bit_ranges,extension
95,addi,"[14..12=0, 6..2=0x04, 1..0=3]",[rv_i]


In [10]:
df_sorted = df.sort_values(by='bit_ranges',key = lambda x: x.apply(len), ascending= False)

In [11]:
df_sorted['bit_ranges'] = df_sorted['bit_ranges'].apply(lambda x: [parse_range(y) for y in x])
df_sorted.head(5)

,instruction,bit_ranges,extension
7,pseudo_op-unzip,"[[31, 25, 100], [24, 20, 1111], [14, 12, 101],...","[rv32_zbkb, rv32_zk, rv32_zkn, rv32_zks]"
6,pseudo_op-zip,"[[31, 25, 100], [24, 20, 1111], [14, 12, 1], [...","[rv32_zbkb, rv32_zk, rv32_zkn, rv32_zks]"
176,pseudo_op-brev8,"[[31, 20, 11010000111], [14, 12, 101], [6, 2, ...","[rv_zbkb, rv_zk, rv_zkn, rv_zks]"
48,pseudo_op-rev8,"[[31, 20, 11010111000], [14, 12, 101], [6, 0, ...","[rv64_zbb, rv64_zbkb, rv64_zk, rv64_zkn, rv64_..."
112,pseudo_op-pause,"[[27, 24, 1], [23, 20, 0], [19, 15, 0], [14, 1...",[rv_i]


In [12]:
df_sorted[df_sorted['instruction'] == 'addi']

,instruction,bit_ranges,extension
95,addi,"[[14, 12, 0], [6, 2, 100], [1, 0, 11]]",[rv_i]


In [13]:
file_path = './tests/memory/000-addi.hex'
with open(file_path, 'r') as file:
    hex_content = file.readlines()

In [14]:
df_sorted[df_sorted['instruction'] == 'addi']['bit_ranges'].to_list()[0][-1:]

[[1, 0, '11']]

In [15]:
hex_content[0].strip()

'00500593'

In [17]:
df_sorted[df_sorted['instruction'] == 'sw']

,instruction,bit_ranges,extension
94,sw,"[[14, 12, 10], [6, 2, 1000], [1, 0, 11]]",[rv_i]


In [18]:
df_sorted.head()

,instruction,bit_ranges,extension
7,pseudo_op-unzip,"[[31, 25, 100], [24, 20, 1111], [14, 12, 101],...","[rv32_zbkb, rv32_zk, rv32_zkn, rv32_zks]"
6,pseudo_op-zip,"[[31, 25, 100], [24, 20, 1111], [14, 12, 1], [...","[rv32_zbkb, rv32_zk, rv32_zkn, rv32_zks]"
176,pseudo_op-brev8,"[[31, 20, 11010000111], [14, 12, 101], [6, 2, ...","[rv_zbkb, rv_zk, rv_zkn, rv_zks]"
48,pseudo_op-rev8,"[[31, 20, 11010111000], [14, 12, 101], [6, 0, ...","[rv64_zbb, rv64_zbkb, rv64_zk, rv64_zkn, rv64_..."
112,pseudo_op-pause,"[[27, 24, 1], [23, 20, 0], [19, 15, 0], [14, 1...",[rv_i]


In [19]:
line = hex_content[0].strip()
print(hex_to_bin(line))
hex_to_bin(line)[::-1]

10100000000010110010011


'11001001101000000000101'

In [21]:
df_sorted[df_sorted['instruction'] == 'sw']['bit_ranges']

94    [[14, 12, 10], [6, 2, 1000], [1, 0, 11]]
Name: bit_ranges, dtype: object

In [25]:
line = hex_content[0].strip()
line = hex_to_bin(line)
line = line[::-1]
print(line)
for rg in df_sorted[df_sorted['instruction'] == 'addi']['bit_ranges'].to_list()[0]:
    if len(rg) == 3:
        end = rg[0]
        start = rg[1]
        value = rg[2]
        print(f"{start} {end}: {value}")        
        inter = line[start:end+1]
        print(inter)
        if int(inter[::-1],2) == int(value,2):
            print("YES")
        
    else:
        position = rg[0]
        value = rg[1]
        print('no')

11001001101000000000101
12 14: 0
000
YES
2 6: 100
00100
YES
0 1: 11
11
YES
